In [1]:
import numpy as np
import pandas as pd
import csv

# 读取 CSV 文件
file_path = "./all_fisher_matrices.csv"  # 更新为实际路径
df = pd.read_csv(file_path)

# 提取数据并构建 Fisher 矩阵
fisher_matrices = {}
for index, row in df.iterrows():
    group = row['Group']
    detector = row['Detector']
    fisher_values = row.iloc[2:].values.astype(float).reshape(9, 9)  # 将数据重塑为9x9矩阵
    key = f"{group}_{detector}"
    fisher_matrices[key] = fisher_values  # 保证矩阵是完整的9维

In [2]:
# 动态正则化项函数
def get_dynamic_epsilon(matrix):
    cond_number = np.linalg.cond(matrix)
    eigvals = np.linalg.eigvals(matrix)
    eigval_std = np.std(eigvals)
    
    if cond_number > 1e20 or eigval_std > 1e2:
        return 1e-2
    elif cond_number > 1e16 or eigval_std > 1e1:
        return 1e-3
    elif cond_number > 1e12:
        return 1e-4
    else:
        return 1e-6

# 确保矩阵正定
def ensure_positive_definite(matrix, matrix_name=""):
    eigvals, eigvecs = np.linalg.eigh(matrix)
    eigvals_clipped = np.clip(eigvals, 1e-2, 1e3)
    matrix_pos_def = eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T
    return matrix_pos_def

# 正则化矩阵
def regularize_matrix(matrix, matrix_name=""):
    epsilon = get_dynamic_epsilon(matrix)
    matrix_sym = (matrix + matrix.T) / 2
    return ensure_positive_definite(matrix_sym + epsilon * np.eye(matrix.shape[0]), matrix_name)

# 计算协方差矩阵
def compute_covariance(fisher_matrix, matrix_name=""):
    reg_matrix = regularize_matrix(fisher_matrix, matrix_name)
    cov_matrix = np.linalg.pinv(reg_matrix)
    return cov_matrix

In [3]:
# 动态调节 KL 散度中的 Trace 和均值差项
def dynamic_clip(value, threshold=250):
    return min(value, threshold)

# 计算 KL 散度
def kl_divergence(cov1, mean1, cov2, mean2, matrix_name=""):
    cov2_regularized = regularize_matrix(cov2, matrix_name)
    eigvals_cov2 = np.linalg.eigvals(cov2_regularized)
    if np.any(eigvals_cov2 <= 0):
        print(f"{matrix_name} has non-positive eigenvalues, skipping KL computation.")
        return np.nan

    try:
        inv_cov2 = np.linalg.inv(cov2_regularized)
    except np.linalg.LinAlgError:
        print(f"{matrix_name} inversion failed, skipping KL computation.")
        return np.nan

    term1 = np.trace(inv_cov2 @ cov1)
    term1 = dynamic_clip(term1)
    print(f"{matrix_name} KL term1 (Trace): {term1}")

    sign1, logdet1 = np.linalg.slogdet(cov1)
    sign2, logdet2 = np.linalg.slogdet(cov2_regularized)
    if sign1 <= 0 or sign2 <= 0:
        print(f"{matrix_name} determinant is non-positive.")
        return np.nan

    term2 = logdet2 - logdet1
    print(f"{matrix_name} KL term2 (Log determinant difference): {term2}")

    term3 = (mean2 - mean1).T @ inv_cov2 @ (mean2 - mean1)
    term3 = dynamic_clip(term3)
    print(f"{matrix_name} KL term3 (Mean difference): {term3}")

    result = 0.5 * (term1 + term2 + term3 - cov1.shape[0])
    print(f"{matrix_name} KL result: {result}")

    return max(result, 0)

# 计算每个 Fisher 矩阵的协方差矩阵和均值
cov_matrices = {}
means = {}
for key, fisher_matrix in fisher_matrices.items():
    cov_matrix = compute_covariance(fisher_matrix, key)
    mean_vector = np.mean(cov_matrix, axis=0)
    cov_matrices[key] = cov_matrix
    means[key] = mean_vector

In [4]:
# 按组计算 KL 散度并保存结果
results = []
groups = df['Group'].unique()

for group in groups:
    group_detectors = [key for key in cov_matrices.keys() if key.startswith(group)]
    if len(group_detectors) >= 3:  # 假设每组有三个探测器
        kl_12 = kl_divergence(cov_matrices[f'{group}_ET_1'], means[f'{group}_ET_1'], 
                              cov_matrices[f'{group}_ET_2'], means[f'{group}_ET_2'], "matrix_12")
        kl_13 = kl_divergence(cov_matrices[f'{group}_ET_1'], means[f'{group}_ET_1'], 
                              cov_matrices[f'{group}_ET_3'], means[f'{group}_ET_3'], "matrix_13")
        kl_23 = kl_divergence(cov_matrices[f'{group}_ET_2'], means[f'{group}_ET_2'], 
                              cov_matrices[f'{group}_ET_3'], means[f'{group}_ET_3'], "matrix_23")

        results.append({
            'group': group,
            'kl_12': kl_12,
            'kl_13': kl_13,
            'kl_23': kl_23,
        })

# 保存结果到CSV文件
output_file = "kl_divergence_results_regularized.csv"
with open(output_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['group', 'kl_12', 'kl_13', 'kl_23'])
    writer.writeheader()
    writer.writerows(results)

print(f"KL divergence results saved to {output_file}")

matrix_12 KL term1 (Trace): 250
matrix_12 KL term2 (Log determinant difference): 9.465740883044745
matrix_12 KL term3 (Mean difference): 243.26570135582074
matrix_12 KL result: 246.86572111943275
matrix_13 KL term1 (Trace): 250
matrix_13 KL term2 (Log determinant difference): 12.680606507766564
matrix_13 KL term3 (Mean difference): 250
matrix_13 KL result: 251.8403032538833
matrix_23 KL term1 (Trace): 250
matrix_23 KL term2 (Log determinant difference): 12.453706943455272
matrix_23 KL term3 (Mean difference): 250
matrix_23 KL result: 251.72685347172762
matrix_12 KL term1 (Trace): 250
matrix_12 KL term2 (Log determinant difference): 7.802855386437269
matrix_12 KL term3 (Mean difference): 250
matrix_12 KL result: 249.40142769321864
matrix_13 KL term1 (Trace): 250
matrix_13 KL term2 (Log determinant difference): 8.32302834523227
matrix_13 KL term3 (Mean difference): 250
matrix_13 KL result: 249.66151417261614
matrix_23 KL term1 (Trace): 250
matrix_23 KL term2 (Log determinant difference):